In [59]:
# 쿠팡 크롤링시 기본 템플릿
import os
import urllib3
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementNotInteractableException
import pandas as pd
import time
from webdriver_manager.chrome import ChromeDriverManager
import requests
from bs4 import BeautifulSoup

os.environ["WDM_SSL_VERIFY"] = "0"

#InsecureRequestWarning 비활성화
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# 크롬 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option("useAutomationExtension", False)
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--disable-popup-blocking")
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.110 Safari/537.36")

# webdriver-manager로 ChromeDriver 자동 설치 및 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options = chrome_options)

# Selenium 감지 속성 제거
driver.execute_cdp_cmd(
    "Page.addScriptToEvaluateOnNewDocument",
    {
        "source": """
        Object.defineProperty(navigator, 'webdriver', {
          get: () => undefined
        })
        """
    }
)

{'identifier': '2'}

In [58]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options = chrome_options)
driver.implicitly_wait(5)

In [ ]:
'''
# url 구성하는 요소
keyword = input('키워드')
official = '&mallTypes=OFFICIAL_CERTIFIED' # 네이버 공식인증
free = '&recommendedDeliveries=FREE_DELIVERY'  # 무료 배송
include = '&includedDeliveryFee=true'  # 배송비 포함
rating = '&score=4.8%7C5'  # ★ 4.8 이상
sort = input('정렬')
minPrice = input('최저 가격')  # 최저 가격
maxPrice = input('최고 가격')  # 최고 가격
priceRange = f'&priceRange={minPrice}%7C{maxPrice}'
url = f'https://search.shopping.naver.com/ns/search?query={keyword}{official}{free}{include}{rating}&sort={sort}{priceRange}'
'''

url = 'https://search.shopping.naver.com/ns/search?query=%EB%8B%AD%EA%B0%80%EC%8A%B4%EC%82%B4&includedDeliveryFee=true&recommendedDeliveries=FREE_DELIVERY'
driver.get(url)
driver.implicitly_wait(5)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

# 크롬 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option("useAutomationExtension", False)
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--disable-popup-blocking")
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.110 Safari/537.36")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options = chrome_options)
url = 'https://search.shopping.naver.com/ns/search?query=%EB%8B%AD%EA%B0%80%EC%8A%B4%EC%82%B4&includedDeliveryFee=true&recommendedDeliveries=FREE_DELIVERY'
driver.get(url)
wait = WebDriverWait(driver, 10)

count = int(input('몇 개? '))
result = []

while len(result) < count:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)  # 페이지 로드를 기다림
    divs = driver.find_elements(By.CSS_SELECTOR, 'div.basicProductCardInformation_basic_product_card_information__7v_uc.basicProductCardInformation_view_type_grid2__mlh6E')
    
    for div in divs:
        if not div.find_elements(By.CSS_SELECTOR, '.basicProductCardInformation_advertisement_area__HzaQ_'):
            name = div.find_element(By.CSS_SELECTOR, 'strong.basicProductCardInformation_title__Bc_Ng').text
            # 데이터 중복 체크
            if name not in [x[1] for x in result]:
                com = div.find_element(By.CSS_SELECTOR, 'span.basicProductCardInformation_mall_name__8IS3Q').text
                price = int(div.find_element(By.CSS_SELECTOR, 'span.priceTag_inner_price__TctbK').text.split('\n')[0].replace(',', ''))
                link = div.find_element(By.CSS_SELECTOR,'a.basicProductCard_link__urzND._nlog_click._nlog_impression_element').get_attribute('href')
                try:
                    star = float(div.find_element(By.CSS_SELECTOR, 'span.productCardReview_star__7iHNO').text.split('\n')[1])
                    review = int(div.find_element(By.CSS_SELECTOR, 'span.productCardReview_text__A9N9N:not(.productCardReview_star__7iHNO)').text.split(' ')[1].replace(',', ''))
                except:
                    star, review = '', ''
                result.append([com, name, price, star, review])

df = pd.DataFrame(result, columns=['업체명','상품명','가격','링크','별점','리뷰 수'])
df.to_excel('abc.xlsx')

# driver.quit()

In [ ]:
a.basicProductCard_link__urzND._nlog_click._nlog_impression_element